<a href="https://colab.research.google.com/github/shluh/ECD01_SupervisedMachineLearning/blob/main/Aula01_classificacao_versao2024_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Disciplina ECD01 - Aprendizado Supervisionado
#### Prof. Anderson Rocha Tavares (artavares@inf.ufrgs.br)

#### *Créditos pelo material: Profa. Mariana Recamonde-Mendoza (http://inf.ufrgs.br/~mrmendoza)*


**Importante:** sempre faça uma cópia do colab (Arquivo -> Salvar uma cópia no drive) ao começar o exercício. Caso contrário, suas alterações não são salvas.





## **Aula 01** - **Tópico: Aprendizado Supervisionado / Classificação**

<br>

**Classificação** é a tarefa de designar um objeto a uma dentre diversas categorias (_i.e._, classes) pré-definidas, com base em seus atributos conhecidos. Em Aprendizado de Máquina (AM), a classificação é realizada por meio de algoritmos de _aprendizado supervisionado_. Estes algoritmos recebem um conjunto de exemplos rotulados (isto é, contendo o par \<entrada, saída esperada\>), e treinam um modelo para aprender um "mapeamento" entre os atributos da entrada e a saída esperada, podendo ser posteriormente utilizado para prever o rótulo de novos exemplos.

<br>

**Objetivo deste notebook**: entender o conceito de aprendizado indutivo e supervisionado, através da solução de uma tarefa simples de classificação.

**Dê dois cliques em uma célula de texto para editar (e.g. para escrever suas respostas)**

<br>

Caso não esteja familiarizado(a) com o uso do Google Colab, recomendo que assista este [vídeo.](https://youtu.be/inN8seMm7UI)

---


*Sobre a nomenclatura usada*: os *exemplos* ou *instâncias* são  sinônimos e se referem a uma entrada - isto é, uma linha do conjunto de dados, representado como uma tabela.

O termo *atributos* se refere às coluna da tabela, que representam as características ou variáveis dos dados de entrada. A coluna que contém a saída esperada será chamada de *atributo alvo* ou *rótulo*.

Para problemas de classificação, o termo *classe* será adotado como um sinônimo para rótulo.


##**Laranja ou limão siciliano?**




Classificar visualmente uma fruta entre laranja e limão siciliano é algo intuitivo para nós (humanos). Como ensinar o computador a fazer esta classificação de forma automática, sem intervenção humana?


Este notebook usará como base um dataset criado pelo [Dr. Iain Murray](https://homepages.inf.ed.ac.uk/imurray2/) (University of Edinburgh), contendo características como a altura (cm), largura (cm) e massa (g) de uma seleção de laranjas e limões sicilianos, quantificadas pelo próprio Dr. Murray. A partir destes dados, discutiremos o processo de criar um classificador para determinar qual a fruta (laranja ou limão siciliano?) com base nas características informadas. Assim, cada fruta é uma instância no dataset, e os atributos são dados pelas respectivas altura, largura e massa.



---

###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Carregando as bibliotecas necessárias
# A primeira linha é incluída para gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline
import pandas as pd             # biblioteca para análise de dados
import matplotlib.pyplot as plt # biblioteca para visualização de informações
import seaborn as sns           # biblioteca para visualização de informações
import numpy as np              # biblioteca para operações com arrays multidimensionais
sns.set()


In [ ]:
fruits = pd.read_table("https://drive.google.com/uc?export=view&id=1wQJ5H0UMZLDj2ZjnbnHUzEnG4YxcQ-cQ")
fruits.head()  # para visualizar apenas as 5 primeiras linhas
## fruits      # para visualizar o dataframe completo

A classe de cada instância está codificada de duas formas, através de um número (*'fruit_label'*) e através de uma string (*'fruit_name'*). Por enquanto manteremos as duas.

Vamos verificar a seguir quais os valores únicos da coluna fruit_name (esperado: orange/lemon) e quantos exemplos temos para cada classe.

In [ ]:
## nome das frutas existentes no arquivo (coluna fruit_name)
print(fruits['fruit_name'].unique())

In [ ]:
## número de exemplos por classe
print(fruits.groupby('fruit_name').size())

Vamos fazer uma breve inspeção destes dados. O método `info()` é útil para obter uma descrição geral do dataframe, como tamanho, índices e nomes das colunas, os respectivos tipos de dados e o número de valores não-nulos.

In [ ]:
fruits.info()

**Responda >>>** Quantas instâncias e quantos atributos existem nos dados? Os atributos possuem valores nulos? São numéricos ou categóricos?

> ***Sua resposta aqui:***


Podemos também analisar a distribuição de valores para atributos **numéricos**, usando o método` describe()`. O atributo *'fruit_label'* é removido da análise pois embora seja um atributo numérico, ele representa a classe do problema e seus valores devem ser interpretados como categorias.

In [ ]:
fruits.drop(['fruit_label'],axis=1).describe()

Uma forma mais fácil e intuitiva de analisar a distribuição de valores para cada atributo é através do uso de histogramas. Para tanto, utilizaremos o método `hist()`. Novamente iremos ignorar a coluna do atributo *'fruit_label'* nesta análise.

In [ ]:
fruits.drop(['fruit_label'],axis=1).hist(bins=15, figsize=(15,10))
plt.show() # este comando é opcional em notebooks, pois os gráficos são automaticamente mostrados quando uma célula é executada

Com o histograma e a sumarização das distribuições dos atributos pelo método `describe()`, já podemos observar algumas características que podem ser relevantes em relação aos dados.


Também podemos fazer a análise da distribuição dos valores dos atributos **por classe**. Execute o exemplo abaixo. Em seguida, adapte o código para fazer a mesma análise para os demais atributos.

In [ ]:
x1 = fruits.loc[fruits.fruit_name=='orange', 'mass']
x2 = fruits.loc[fruits.fruit_name=='lemon', 'mass']

kwargs = dict(alpha=0.5, bins=10) # parametros de visualizacao do histograma

plt.hist(x1, **kwargs, color='g', label='Orange')
plt.hist(x2, **kwargs, color='b', label='Lemon')
plt.gca().set(title='Frequency Histogram', ylabel='Frequency')
plt.legend()


**Responda >>>** Com o código acima, faça a análise para os quatro atributos existentes. Algum destes atributos parece ter um maior potencial na tarefa de distinguir uma nova fruta entre laranja e limão siciliano? Comente a respeito.



> ***Sua resposta aqui:***



---


### Visualizando os dados

Para discutirmos a ideia de classificação e entendermos o seu funcionamento com este dataset, vamos assumir que o vetor de atributos que descreve cada um dos exemplos disponíveis tem apenas duas dimensões: **altura (height)** e **largura (width)**. Podemos facilmente visualizar nosso problema de classificação no seu espaço de entrada, seguindo exemplo discutido em aula.

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='width', y='height', data=fruits, hue='fruit_name', s=60)
plt.legend(loc='lower right', title='Fruit')
plt.show()

**Responda >>>** Este problema de classificação é linearmente separável?



> ***Sua resposta aqui:***


---


### Definindo uma fronteira de decisão linear

Para o problema de classificação *Laranja ou limão siciliano?*, vamos assumir que desejamos definir manualmente uma fronteira de decisão linear a partir dos atributos de **altura** e **largura**.

O gráfico abaixo é interativo e ele possui sliders de controle para ajustar os dois coeficientes de uma equação reduzida da reta
$ax + b$, onde $a$ é a inclinação da reta e $b$ o  intercepto. A partir da determinação de a e b, podemos definir uma fronteira de decisão linear representando o nosso classificador. A partir da fronteira de decisão, a região ganha o rótulo da classe majoritária nela.

**Experimente >>>** Faça ajustes nos sliders até encontrar uma fronteira de decisão que você julgue separar bem as classes. Informe os valores de $a$ e $b$ da sua equação.

> ***Sua resposta aqui:***

In [ ]:
%matplotlib inline
from ipywidgets import interactive


def plot_interactive1(a=0, b=8.5):
    x = np.linspace(5, 10.5, num=1000)
    plt.figure(figsize=(8, 6))
    g=plt.scatter(x='width', y='height', data=fruits, c='fruit_label',cmap="Accent",s=45)
    plt.plot(x, a * x + b)
    plt.ylim(6, 11)
    plt.xlabel('Width')
    plt.ylabel('Height')
    plt.legend(handles=g.legend_elements()[0],labels=('orange','lemon'))

    plt.show()

interactive_plot = interactive(plot_interactive1, a=(-3.0, 3.0,0.05), b=(-12, 12, 0.05))
output = interactive_plot.children[-1]
output.layout.height = '400px'
interactive_plot


**Experimente >>>** No código abaixo, customize os valores de $a$ e $b$ de acordo com a sua função da fronteira de decisão (exercício anterior). Você pode utilizar o formulário à direita para ajustar estes valores. Em seguida, determine qual classe seria retornada pelo modelo (representado pela sua fronteira de decisão) para os novos dados informados.
**OBS.:** São três novas instâncias que precisam ser analisadas. Descomente uma de cada vez, execute o código, analise a posição da instância e responda a questão.

> ***Sua resposta aqui:***

In [ ]:
## Definindo entrada de valores a partir do form para os coeficientes da equação
a =  0#@param {type:"number"}
b =  8.5#@param {type:"number"}

## Nas linhas abaixo, descomente uma por vez para visualizar a projeção do novo ponto
## no espaço de entrada. Após definir a classe predita pelo modelo, comenta a linha
## analisada e descomente a próxima. Você irá visualizar e analisar um ponto de cada vez.
new_data = [7.5,10.5] ## novo dado #1
#new_data = [6.65,7.6] ## novo dado #2
#new_data = [6.0,7.0] ## novo dado #3


def plot_newdatapoint(a=a,b=b,xt=new_data[0],yt=new_data[1]):
    x = np.linspace(5, 10.5, num=1000)
    plt.figure(figsize=(8, 6))
    g=plt.scatter(x='width', y='height', data=fruits, c='fruit_label',cmap="Accent",s=45)
    plt.plot(x, a * x + b)
    plt.plot(xt,yt,'ro')
    plt.ylim(6, 11)
    plt.xlabel('Width')
    plt.ylabel('Height')
    plt.legend(handles=g.legend_elements()[0],labels=('orange','lemon'))

plot_newdatapoint()


---


### Fronteiras de decisão não-lineares

A grande maioria dos problemas de classificação  possuem demanda fronteiras não-lineares.

Veja, por exemplo, o problema de classificação abaixo, no qual idade e renda são utilizados como preditores para o tipo de serviço adquirido (básico ou premium). A imagem foi extraída [desta fonte](https://docs.microsoft.com/pt-br/azure/machine-learning/how-to-select-algorithms).


<img src="https://drive.google.com/uc?export=view&id=1w_uNHllvFga79WCeOtCrv6gjHx-0o3jT"
    style="width: 700px; max-width: 100%; height: auto"/>
  

A partir da próxima aula vamos começar a discutir algoritmos de aprendizado supervisionado que são capazes de gerar fronteiras de decisão não-lineares.

## Instruções para a entrega das atividades práticas:

Todos os exercícios da disciplina serão desenvolvidos no Google Colab. Ao final de cada atividade prática,  você deverá entregar as suas respostas para as atividades propostas ao longo da atividade. A entrega será baseada na exportação do seu notebook e será feita pelo link indicado no Moodle. Você deve sempre o arquivo `.ipynb`, que pode ser exportado desse colab da seguinte forma:

*   File > Download .ipynb



